In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

| | |
|-|-|
|Author(s) | [Lavi Nigam](https://github.com/lavinigam-gcp) |


**bit.ly / io24-gemini-api**

## What do we want to build?



![](https://storage.googleapis.com/gemini-lavi-asset/img/output.png)

# Step 1 - Google Cloud Vertex AI Gemini API

In [1]:
# Library installation # needs restarts
! pip install --upgrade google-cloud-aiplatform
! pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 11.4 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.48.0
    Uninstalling google-cloud-aiplatform-1.48.0:
      Successfully uninstalled google-cloud-aiplatform-1.48.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.2 MB/s eta 0:00:00


In [1]:
# Authentication

import sys
from google.cloud import storage

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = ""  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}
    BUCKET_NAME = "gemini-lavi-asset" # @param {type:"string"}
    # Initialize Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)

    # Initialize cloud storage
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(BUCKET_NAME)

In [2]:
# Library
from IPython.display import display, Markdown, Latex
import vertexai.generative_models as genai
import PyPDF2
import pandas as pd
import time
from vertexai.generative_models import (
    GenerativeModel,
    Part,
    GenerationConfig,
    HarmCategory,
    HarmBlockThreshold,
)
import PyPDF2
from io import BytesIO
from vertexai.preview.generative_models import Part
from google.cloud import storage
from io import BytesIO
from datetime import datetime, timezone
import numpy as np
import numpy as np
from typing import List
from IPython.display import display, Markdown, Latex
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
from vertexai.generative_models import (
    GenerativeModel,
    Part,
    GenerationConfig,
    HarmCategory,
    HarmBlockThreshold,
)
from typing import List
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
from google.cloud import storage
from io import BytesIO
from rich import print as rich_print
from rich.markdown import Markdown as rich_Markdown
import pickle
pd.options.mode.chained_assignment = None

## Building Metadata & Index

In [3]:
gemini_15_pro_new = genai.GenerativeModel("gemini-1.5-pro-preview-0514")
gemini_15_flash = genai.GenerativeModel("gemini-1.5-flash-preview-0514")

In [4]:
#@title Helper Functions

def get_gemini_response(model, generation_config=None,
                        safety_settings=None,
                        uri_path=None,mime_type=None, prompt=None):
  if not generation_config:
    generation_config = {
      "max_output_tokens": 8192,
      "temperature": 1,
      "top_p": 0.95,
    }

  if not safety_settings:
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

  uri = "gs://"+uri_path
  file = genai.Part.from_uri(
    mime_type=mime_type,
    uri=uri
    )
  responses = model.generate_content([file, prompt],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )
  final_response = []
  for response in responses:
    try:
      final_response.append(response.text)
    except ValueError:
      # print("Something is blocked...")
      final_response.append("blocked")

  return "".join(final_response)

def get_text_from_pdf(bucket):
  extracted_text = []
  # Iterate over all blobs (files) in the bucket
  for blob in bucket.list_blobs():
      if blob.name.startswith("production/"):
        if blob.name.lower().endswith('.pdf'):  # Check if the file is a PDF
            # Download the PDF to a BytesIO object
            pdf_content = BytesIO(blob.download_as_bytes())
            try:
                # Process the PDF using PyPDF2
                pdf_reader = PyPDF2.PdfReader(pdf_content)
                text = ""
                pdf_data = []
                text_type = "/".join(blob.name.split("/")[1:-1])
                filename = blob.name.split("/")[-1]
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    text = page.extract_text()
                    if text:
                      pdf_data.append(
                      {
                          'text_type' : text_type,
                          'gcs_path': "gs://"+blob.bucket.name+"/"+blob.name,
                          'page_number': page_num+1,
                          'text': text
                      }
                      )
                extracted_text.extend(pdf_data)
                # break
            except:
                print(f"Warning: Could not read PDF file '{blob.name}' (might be encrypted or corrupted)")
  return pd.DataFrame(extracted_text)

# 16-17 min
def get_text_from_video(bucket, model, prompt, time_sleep=5):
  video_metadata = []
  for blob in bucket.list_blobs():
      if blob.name.startswith("production/"):
        if blob.name.lower().endswith('.mp4'):
          print("processing....",blob.name)
          video_type = "/".join(blob.name.split("/")[1:-1])
          gcs_path = "/".join(blob.id.split("/")[:-1])
          try:
            video_description = get_gemini_response(uri_path=gcs_path,
                                                    model=model,
                                                    mime_type='video/mp4',
                                                    prompt = prompt)
            if video_description:
              video_metadata.append(
                  {
                      'video_gcs': "gs://"+blob.bucket.name+"/"+blob.name,
                      'video_type':video_type,
                      'video_description': video_description
                  }

              )
          except:
            print("Something Failed........")
            video_metadata.append(
                  {
                      'video_gcs': "gs://"+blob.name,
                      'video_type':video_type,
                      'video_description': ""
                  }

              )
          # print("sleeping......")
          time.sleep(time_sleep)
          # break
  return pd.DataFrame(video_metadata)

# ~20 min
def get_text_from_audio(bucket, model, prompt, time_sleep=5):
  # Iterate over all blobs (files) in the bucket
  audio_metadata = []
  for blob in bucket.list_blobs():
    if blob.name.startswith("production/"):
      if blob.name.lower().endswith('.mp3'):
        print("processing....",blob.name)
        video_type = "/".join(blob.name.split("/")[1:-1])
        gcs_path = "/".join(blob.id.split("/")[:-1])
        # print(gcs_path)
        try:
          audio_description = get_gemini_response(uri_path=gcs_path,
                                                  model=model,
                                                  mime_type='audio/mpeg',
                                                  prompt=prompt)
          if audio_description:
            audio_metadata.append(
                {
                    'audio_gcs': "gs://"+blob.bucket.name+"/"+blob.name,
                    'audio_type':video_type,
                    'audio_description': audio_description
                }

            )
        except:
          print("Something Failed........")
          audio_metadata.append(
              {
                  'audio_gcs': blob.name,
                  'audio_type':video_type,
                  'audio_description': ""
              }

          )
        # print("sleeping......")
        time.sleep(time_sleep)
        # break
  return pd.DataFrame(audio_metadata)

def split_text_into_chunks(df, text_column, chunk_size):
    """Splits text into chunks of specified size, preserving other column values."""

    # Create a list of new dataframes, one for each chunk
    new_dfs = []
    for _, row in df.iterrows():
        text_chunks = [row[text_column][i:i + chunk_size] for i in range(0, len(row[text_column]), chunk_size)]
        for chunk in text_chunks:
            new_row = row.copy()  # Copy all other columns
            new_row[text_column] = chunk
            new_dfs.append(pd.DataFrame([new_row]))

    return pd.concat(new_dfs, ignore_index=True)  # Combine into single dataframe

def get_text_embeddings(
    texts: List[str] = ["banana muffins? ", "banana bread? banana muffins?"],
    task: str = "RETRIEVAL_DOCUMENT",
    model_name: str = "textembedding-gecko@003",
) -> List[List[float]]:
    # print("doing...")
    """Embeds texts with a pre-trained, foundational model."""
    model = TextEmbeddingModel.from_pretrained(model_name)
    inputs = [TextEmbeddingInput(text, task) for text in texts]
    embeddings = model.get_embeddings(inputs)
    return [embedding.values for embedding in embeddings][0]

def backup_metadata_in_pickle(extracted_text,
                              video_metadata,
                              audio_metadata,
                              index_db,
                              output_path_with_name):
  import pickle
  data_to_save = {
      "extracted_text": extracted_text,
      "video_metadata": video_metadata,
      "audio_metadata": audio_metadata,
      "index_db": index_db
  }
  print("Backing up the metadata in: ",output_path_with_name+".pkl")
  with open(f"{output_path_with_name}.pkl", "wb") as f:
      pickle.dump(data_to_save,f)

# def load_backuped_metadata(backup_path_with_name):
#   import pickle
#   import pandas as pd

#   with open(f"{backup_path_with_name}.pkl", "rb") as f:
#       loaded_data = pickle.load(f)

#   extracted_text = loaded_data["extracted_text"]
#   video_metadata = loaded_data["video_metadata"]
#   audio_metadata = loaded_data["audio_metadata"]
#   index_db = loaded_data["index_db"]
#   return extracted_text, video_metadata, audio_metadata, index_db

def load_backuped_metadata(bucket):
    for blob in bucket.list_blobs():
      if blob.name.startswith("asset/"):
        if blob.name.lower().endswith('.pkl'):
          with blob.open("rb") as f:
            loaded_data = pickle.load(f)

    extracted_text = loaded_data["extracted_text"]
    video_metadata = loaded_data["video_metadata"]
    audio_metadata = loaded_data["audio_metadata"]
    index_db = loaded_data["index_db"]

    return extracted_text, video_metadata, audio_metadata, index_db


## Step 1 - Text Extraction

### From all reports

![](https://storage.googleapis.com/gemini-lavi-asset/img/Step1.png)

In [24]:
%%time
extracted_text = get_text_from_pdf(bucket)

CPU times: user 2min 8s, sys: 693 ms, total: 2min 8s
Wall time: 2min 15s


In [25]:
extracted_text.head()

,text_type,gcs_path,page_number,text
0,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,1,Google\nCloud\ndemonstrates\nthe\nworld’ s\nla...
1,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,2,Source:\n“Computation\nused\nto\ntrain\nnotabl...
2,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,3,1.\nOrchestration:\nThe\nsoftware\nstack\nthat...
3,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,4,Cloud\nTPU\nMultislice\nTraining\nhas\nthe\nfo...
4,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,5,1.\nRobust\norchestration\nand\nscalability:\n...


In [26]:
print("Pages per file...",
      extracted_text['gcs_path'].value_counts())

gcs_path
gs://gemini-lavi-asset/production/reports/2021/annual_report/2021-alphabet-annual-report.pdf         136
gs://gemini-lavi-asset/production/reports/2020/annual_report/2020-alphabet-annual-report.pdf         132
gs://gemini-lavi-asset/production/reports/2022/annual_report/2022-alphabet-annual-report.pdf         129
gs://gemini-lavi-asset/production/reports/2020/quaterly_report/20210203-alphabet-10k.pdf             100
gs://gemini-lavi-asset/production/reports/2023/quaterly_report/goog-10-k-2023.pdf                     97
gs://gemini-lavi-asset/production/reports/2021/quaterly_report/20220202-alphabet-10k.pdf              95
gs://gemini-lavi-asset/production/reports/2022/quaterly_report/goog-10-k-q4-2022.pdf                  92
gs://gemini-lavi-asset/production/reports/2020/quaterly_report/20201030-alphabet-10q.pdf              73
gs://gemini-lavi-asset/production/reports/2020/quaterly_report/20200731-alphabet-10q.pdf              70
gs://gemini-lavi-asset/production/reports/2020

In [28]:
print("total files: ....",
      len(extracted_text['gcs_path'].value_counts().index))

total files: .... 32


In [29]:
print("total pages: ....",
      sum(extracted_text['gcs_path'].value_counts().values))

total pages: .... 1648


In [58]:
rich_Markdown(extracted_text['text'][9])

Storage Interacting with persistent storage is a crucial aspect of training. Our 199-pod cluster had 1 Tb/s to     
Cloud Storage, 1,270 Tb/s inter-slice DCN, and 73,400 Tb/s intra-slice ICI. When loading Docker images, loading    
data, and reading/writing checkpoints, we optimized the interaction with persistent storage. We found that at large
scale, data loading from Cloud Storage began to affect performance, starting at 64 pods scale. We have since       
mitigated this limit with a distributed data loading strategy that alleviates pressure on Cloud Storage by having a
subset of hosts load data. We also found limits due to checkpointing. By default, checkpointing loads the full     
checkpoint into each data parallel replica from Cloud Storage. Consider checkpoint loading for a 128B model sharded
with cross-pod data parallelism. For a traditional optimizer state of three numbers per parameter (4bytes/number), 
this means loading a checkpoint of size ~1.536 TB separately into each pod (in this case for 199 pods). This would 
require 199 pods * 1.536TB/pod, or approximately 300TB of bandwidth. For reasonable performance from persistent    
storage of 1 Tb/s, this would require approximately 2,400 seconds (40 minutes). However, we needed much lower start
or restart time, so had to take a different approach.

### From audio files

![](https://storage.googleapis.com/gemini-lavi-asset/img/Step1-Audio.png)

In [42]:
%%time

audio_description_extraction_prompt = """Transcribe and analyze the audio, identifying key topic shifts or changes in focus. Divide the audio into segments based on these transitions.
For each segment:
* **Summarize:** Briefly describe the main topic or theme of the segment.
* **Contextualize:** Explain how this topic fits into the broader conversation or narrative.
* **Analyze:** Explore the significance of this topic, the perspectives presented, and any potential biases or underlying assumptions.
* **Synthesize:** Connect this topic to other themes or ideas mentioned in the audio, highlighting relationships and overarching patterns.
Conclude with a thematic analysis of the entire audio. Identify the most prominent themes, how they are interconnected, and the overall message or purpose of the audio.
"""

audio_metadata = get_text_from_audio(bucket, gemini_15_pro_new,
                                     audio_description_extraction_prompt,
                                    )

processing.... production/earning_calls/2023 Annual Meeting of Stockholders.mp3
processing.... production/earning_calls/Alphabet 2023 Q1 Earnings Call.mp3
processing.... production/earning_calls/Alphabet 2023 Q2 Earnings Call.mp3
processing.... production/earning_calls/Alphabet 2023 Q3 Earnings Call.mp3
processing.... production/earning_calls/Alphabet 2023 Q4 Earnings Call.mp3
processing.... production/podcast/dialogue_dispatch/Can AI help us accelerate a green economy_ _ Dialogues Dispatch Podcast _ Ep 5 Trailer.mp3
processing.... production/podcast/dialogue_dispatch/Can AI help us accelerate a green economy_ _ Dialogues Dispatch Podcast _ Episode 5.mp3
processing.... production/podcast/dialogue_dispatch/Dialogues Dispatch Podcast _ Series Trailer.mp3
processing.... production/podcast/dialogue_dispatch/How can AI help accelerate progress on the UN SDGs_ _ Dialogues Dispatch Podcast _ Ep 6 Trailer.mp3
processing.... production/podcast/dialogue_dispatch/How can AI help accelerate progre

In [44]:
# %%time
# audio_metadata_flash = get_text_from_audio(bucket, gemini_15_flash,
#                                      audio_description_extraction_prompt,
#                                     )

In [46]:
audio_metadata.tail()

,audio_gcs,audio_type,audio_description
13,gs://gemini-lavi-asset/production/podcast/dial...,podcast/dialogue_dispatch,## Analysis of Dialogues Dispatch Podcast\n\nH...
14,gs://gemini-lavi-asset/production/podcast/dial...,podcast/dialogue_dispatch,## Analysis of Dialogues Dispatch Podcast Segm...
15,gs://gemini-lavi-asset/production/podcast/dial...,podcast/dialogue_dispatch,## Audio Transcription Analysis \n\n**Segment ...
16,gs://gemini-lavi-asset/production/podcast/dial...,podcast/dialogue_dispatch,## Audio Segment Analysis\n\n**Segment 1: Intr...
17,gs://gemini-lavi-asset/production/podcast/dial...,podcast/dialogue_dispatch,## Segment 1: Defining Generative AI (0:00 - 4...


In [48]:
print("total files: ....",
      len(audio_metadata['audio_gcs'].value_counts().index))

total files: .... 18


In [51]:
rich_Markdown(audio_metadata['audio_description'][2])

Thematic Analysis of Alphabet Earnings Call Q2 2023                                

This audio is a transcript of Alphabet's second quarter 2023 earnings conference call. It provides an overview of  
the company's financial performance and strategic priorities, with a particular emphasis on advancements in        
artificial intelligence (AI).                                                                                      

Segment 1: Introduction and Safe Harbor                                                                            

 • Summarize: The segment begins with a standard welcome and operator instructions for the Q&A session.  Jim       
   Friedland, Director of Investor Relations, then provides a safe harbor statement, cautioning listeners about    
   forward-looking statements.                                                                                     
 • Contextualize: This is a necessary legal formality for publicly traded companies during earnings calls.         
 • Analyze: The safe harbor statement highlights the inherent uncertainty associated with future predictions and   
   emphasizes the potential for actual results to differ from projected outcomes.                                  
 • Synthesize: This segment sets the stage for the call by outlining the format and legal parameters within which  
   the company will be discussing its performance and plans.                                                       

Segment 2: CEO Sundar Pichai's Opening Remarks                                                                     

 • Summarize: Sundar Pichai highlights Alphabet's recent events like Google I/O, Brandcast, and Google Marketing   
   Live, emphasizing the company's strong momentum. He acknowledges solid performance in Search and YouTube, along 
   with sustained growth in Google Cloud. He also points to ongoing efforts towards operational efficiency and     
   durable savings to fund investment in key priorities, particularly AI.                                          
 • Contextualize: This segment provides an optimistic initial assessment of the quarter's performance and sets the 
   agenda for the following discussion by focusing on AI's central role in Alphabet's strategy.                    
 • Analyze: Pichai’s focus on AI reflects the company's commitment to leading the development and application of   
   this technology across its product portfolio. The emphasis on efficiency and savings suggests a strategic       
   response to economic uncertainty and the need to prioritize investments in areas like AI with long-term growth  
   potential.                                                                                                      
 • Synthesize:  This segment reinforces AI as the overarching theme of the call, establishing it as the key driver 
   of both Alphabet's present performance and its future ambitions.                                                

Segment 3:  Focus on AI:  Making It Helpful for Everyone                                                           

 • Summarize: Pichai outlines four key areas where Alphabet is leveraging AI to benefit users: improving knowledge 
   and learning, boosting creativity and productivity, facilitating innovation by others, and ensuring responsible 
   development and deployment. He discusses the evolution of Search with the Search Generative Experience (SGE),   
   powered by generative AI, and user's positive response to this new capability.                                  
 • Contextualize: This segment dives deep into Alphabet's AI initiatives, showcasing the breadth and impact of the 
   company's work in this field.                                                                                   
 • Analyze: This section demonstrates how AI is not just a buzzword for Alphabet but a deeply integrated aspect of 
   their product development and user experience strategy. The four key areas pro

### From video files

![](https://storage.googleapis.com/gemini-lavi-asset/img/Step1_Video.png)

In [59]:
%%time

video_description_extraction_prompt = """Transcribe and analyze the video, intelligently segmenting it based on shifts in topic, focus, or narrative progression.
For each identified segment:
**Concise Summary**: Distill the core theme or message in 1-2 sentences.
**Thematic Context**: How does this segment contribute to the overarching narrative or argument?
**Critical Analysis**: Delve into the segment's implications, perspectives presented, and potential biases.
**Connections**: Link this segment to other parts of the video, revealing patterns and relationships.

Conclude by synthesizing the video's main themes, their interconnections, and the overarching purpose or message.
"""

video_metadata = get_text_from_video(bucket,gemini_15_pro_new,
                                    video_description_extraction_prompt,
                                    )


processing.... production/product_launch/gemini/Can AI understand your outfit  Testing Gemini.mp4
processing.... production/product_launch/gemini/Can_AI_understand_new_emojis_Testing_Gemini.mp4
processing.... production/product_launch/gemini/Converting images into code with AI  Testing Gemini.mp4
processing.... production/product_launch/gemini/Finding connections with AI  Testing Gemini.mp4
processing.... production/product_launch/gemini/Googles newest AI in 90 seconds  Gemini.mp4
processing.... production/product_launch/gemini/Googles newest and most capable AI  Gemini.mp4
processing.... production/product_launch/gemini/Guessing movies with AI  Testing Gemini.mp4
processing.... production/product_launch/gemini/Math & physics with AI  Gemini.mp4
processing.... production/product_launch/gemini/Multimodal prompting with a 44-minute movie  Gemini 15 Pro Demo.mp4
processing.... production/product_launch/gemini/Personalized AI for you  Gemini.mp4
processing.... production/product_launch/gem

In [61]:
%%time

video_metadata_flash = get_text_from_video(bucket,gemini_15_flash,
                                    video_description_extraction_prompt,
                                    )

processing.... production/product_launch/gemini/Can AI understand your outfit  Testing Gemini.mp4
processing.... production/product_launch/gemini/Can_AI_understand_new_emojis_Testing_Gemini.mp4
processing.... production/product_launch/gemini/Converting images into code with AI  Testing Gemini.mp4
processing.... production/product_launch/gemini/Finding connections with AI  Testing Gemini.mp4
processing.... production/product_launch/gemini/Googles newest AI in 90 seconds  Gemini.mp4
processing.... production/product_launch/gemini/Googles newest and most capable AI  Gemini.mp4
processing.... production/product_launch/gemini/Guessing movies with AI  Testing Gemini.mp4
processing.... production/product_launch/gemini/Math & physics with AI  Gemini.mp4
processing.... production/product_launch/gemini/Multimodal prompting with a 44-minute movie  Gemini 15 Pro Demo.mp4
processing.... production/product_launch/gemini/Personalized AI for you  Gemini.mp4
processing.... production/product_launch/gem

In [60]:
video_metadata.head()

,video_gcs,video_type,video_description
0,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,## Video Analysis: Can Gemini Understand Outfi...
1,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,## Video Segmentation and Analysis:\n\n**Segme...
2,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,## Video Transcript and Analysis:\n\n**Segment...
3,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,## Video Analysis\n\n### Segment 1: Introducti...
4,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,## Gemini: A Multimodal AI Revolution\n\nThe v...


In [62]:
print("total files: ....",
      len(video_metadata['video_gcs'].value_counts().index))

total files: .... 18


In [63]:
rich_Markdown(video_metadata['video_description'][0])

Video Analysis: Can Gemini Understand Outfits?                                   

The video showcases a series of visual reasoning tests conducted on Google's Gemini, a multimodal AI model.        

Segment 1: Puffy Jacket in the Tundra (0:00 - 0:19)                                                                

 • Concise Summary: Gemini is shown a puffy jacket and asked what someone wearing it would be best dressed to do.  
   Gemini responds with a logically sound but humorous answer: staying warm in the tundra.                         
 • Thematic Context: This segment introduces the core concept: testing Gemini's ability to reason about outfits and
   their contextual relevance.                                                                                     
 • Critical Analysis: Gemini's response demonstrates an understanding of the jacket's functional purpose (warmth)  
   and a creative ability to associate it with a suitable environment (tundra). However, the humorous element      
   (blending with glacial ice) highlights that Gemini might be drawing connections based on learned associations   
   rather than a nuanced understanding of social contexts.                                                         
 • Connections: This segment sets the stage for subsequent tests that progressively increase in complexity.        

Segment 2: Intergalactic Jazz Musician (0:19 - 0:38)                                                               

 • Concise Summary:  An astronaut helmet and a saxophone are added to the puffy jacket. Gemini accurately          
   identifies the outfit as suitable for intergalactic travel and then coins the term "Mooncore" for the style.    
 • Thematic Context: This segment builds upon the initial test, demonstrating Gemini's capacity to process multiple
   visual cues and synthesize a coherent interpretation.                                                           
 • Critical Analysis: Gemini's ability to coin a plausible term like "Mooncore" signifies its potential for        
   creativity and understanding of fashion trends. However, the humor inherent in the scenario (spacefaring jazz   
   musician) raises questions about the model's grasp of real-world social norms.                                  
 • Connections: This segment reinforces the playful and experimental tone of the video, while simultaneously       
   showcasing the evolving capabilities of Gemini.                                                                 

Segment 3: Visual Tests Conclusion (0:38 - 0:44)                                                                   

 • Concise Summary: The video concludes by acknowledging Gemini's capability to "understand outfits" and teases    
   further visual tests to come.                                                                                   
 • Thematic Context:  This segment solidifies the video's objective: presenting a glimpse into Gemini's visual     
   reasoning skills, specifically within the domain of fashion and social context.                                 
 • Critical Analysis: The open-ended conclusion emphasizes the ongoing nature of Gemini's development and the      
   potential for further exploration of its multimodal capabilities.                                               
 • Connections: This segment connects back to the initial question posed in the video title, leaving the audience  
   intrigued about the future possibilities of Gemini.                                                             


                                         Synthesis and Overarching Message                                         

The video presents a light-hearted yet insightful exploration of Gemini's ability to understand outfits.  Through a
series of increasingly complex visual tests, Gemini demonstrates its capacity for:                                 

 1 Functional Reasoning: Recognizing the purpose of garments (e.g., warmth).   

## Step 2 - Text Chunking

<img src="https://storage.googleapis.com/gemini-lavi-asset/img/Step2-Chunking.png" width="500" />

In [65]:
# Step 2 [Why do we still do chunking? Explain ----
# 1) show the token count
# 2) reduce noise while search  ]
# latency and cost consideriation - you can still do that, but would it make sense
# Out of 5M token "information" -> you would still want to makes ure that the 1M that you send are the most relevant 1M
# Chunking the Text to smaller size to make precise match with queries

chunk_size =500
extracted_text_chunk_df = split_text_into_chunks(extracted_text, 'text', chunk_size)
video_metadata_chunk_df = split_text_into_chunks(video_metadata, 'video_description', chunk_size)
audio_metadata_chunk_df = split_text_into_chunks(audio_metadata, 'audio_description', chunk_size)

In [66]:
extracted_text_chunk_df.head()

,text_type,gcs_path,page_number,text
0,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,1,Google\nCloud\ndemonstrates\nthe\nworld’ s\nla...
1,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,2,Source:\n“Computation\nused\nto\ntrain\nnotabl...
2,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,3,1.\nOrchestration:\nThe\nsoftware\nstack\nthat...
3,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,3,e\nscale\nrequires\ndeep\nexpertise\nthroughou...
4,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,3,"scalable,\nreliable,\nand\neasy-to-use\nfor\ne..."


In [78]:
rich_Markdown(
    extracted_text[
        (extracted_text["page_number"] == 3)
        & (
            extracted_text["gcs_path"]
            == "gs://gemini-lavi-asset/production/blogpost/Google Cloud TPU blog.pdf"
        )
    ]["text"].values[0]
)


Orchestration: The software stack that is used for distributed training needs to manage all of these chips and     
scale as high as possible to accelerate training times. This stack also needs to be reliable, fault tolerant, and  
resilient in order to ensure training progress. 2. Compilation : As training progresses, the computation and       
communication that happen across the chips need to be managed effectively by a high-performance compiler. 3.       
End-to-end optimization : Distributed training at large scale requires deep expertise throughout both the ML       
training stack and the end-to-end ML training workflow, from storage and compute to memory and networking. Google  
Cloud TPU Multislice Training To address each of the above distributed training challenges across orchestration,   
compilation, and end-to-end optimization, today we announced the general availability of Cloud TPU Multislice      
Training. This full-stack training offering — supporting TPU v4 and v5e — is built from the ground up to be        
scalable, reliable, and easy-to-use for end-to-end optimization of ML training. With Multislice, you can leverage  
Google’s cost-efficient, versatile, and scalable Cloud TPUs for training ML models efficiently and at large scale.

In [81]:
rich_Markdown(extracted_text_chunk_df.iloc[2]['text'])

Orchestration: The software stack that is used for distributed training needs to manage all of these chips and     
scale as high as possible to accelerate training times. This stack also needs to be reliable, fault tolerant, and  
resilient in order to ensure training progress. 2. Compilation : As training progresses, the computation and       
communication that happen across the chips need to be managed effectively by a high-performance compiler. 3.       
End-to-end optimization : Distributed training at larg

In [82]:
rich_Markdown(extracted_text_chunk_df.iloc[3]['text'])

e scale requires deep expertise throughout both the ML training stack and the end-to-end ML training workflow, from
storage and compute to memory and networking. Google Cloud TPU Multislice Training To address each of the above    
distributed training challenges across orchestration, compilation, and end-to-end optimization, today we announced 
the general availability of Cloud TPU Multislice Training. This full-stack training offering — supporting TPU v4   
and v5e — is built from the ground up to be

In [83]:
rich_Markdown(extracted_text_chunk_df.iloc[4]['text'])

scalable, reliable, and easy-to-use for end-to-end optimization of ML training. With Multislice, you can leverage  
Google’s cost-efficient, versatile, and scalable Cloud TPUs for training ML models efficiently and at large scale.

## Step 3 - Embedding



<img src="https://storage.googleapis.com/gemini-lavi-asset/img/Step2-Chunking_embedding.png" width="500" />

In [ ]:
%%time
# Step 3
# Building Embeddings of the text

extracted_text_chunk_df['embeddings'] = extracted_text_chunk_df['text'].apply(lambda x: get_text_embeddings([x]))
video_metadata_chunk_df['embeddings'] = video_metadata_chunk_df['video_description'].apply(lambda x: get_text_embeddings([x]))
audio_metadata_chunk_df['embeddings'] = audio_metadata_chunk_df['audio_description'].apply(lambda x: get_text_embeddings([x]))

In [86]:
extracted_text_chunk_df.head()

,gcs_path,page_number,text,embeddings
0,gs://gemini-lavi-asset/production/blogpost/Goo...,1,Google\nCloud\ndemonstrates\nthe\nworld’ s\nla...,"[0.0026181433349847794, -0.006980935577303171,..."
1,gs://gemini-lavi-asset/production/blogpost/Goo...,2,Source:\n“Computation\nused\nto\ntrain\nnotabl...,"[0.02464565820991993, -0.020231368020176888, -..."
2,gs://gemini-lavi-asset/production/blogpost/Goo...,3,1.\nOrchestration:\nThe\nsoftware\nstack\nthat...,"[0.020969916135072708, 0.0036780794616788626, ..."
3,gs://gemini-lavi-asset/production/blogpost/Goo...,3,e\nscale\nrequires\ndeep\nexpertise\nthroughou...,"[0.012977363541722298, -0.02937842719256878, -..."
4,gs://gemini-lavi-asset/production/blogpost/Goo...,3,"scalable,\nreliable,\nand\neasy-to-use\nfor\ne...","[0.052400510758161545, -0.06389233469963074, -..."


In [89]:
video_metadata_chunk_df.head()

,video_gcs,video_type,video_description,embeddings
0,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,## Video Analysis: Understanding Multimodal Mo...,"[0.01595981791615486, -0.04110850393772125, -0..."
1,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,ike a puffy jacket establishes a baseline for ...,"[0.0016808598302304745, -0.02568306028842926, ..."
2,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,**Thematic Context**: Illustrates Gemini's a...,"[0.025427915155887604, -0.050456706434488297, ..."
3,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,\n\n**Segment 3: Introducing New Elements and...,"[0.03876801207661629, -0.024181410670280457, -..."
4,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,actical to imaginative responses underscores t...,"[0.04932378977537155, -0.03003864549100399, -0..."


In [90]:
audio_metadata_chunk_df.head()

,audio_gcs,audio_type,audio_description,embeddings
0,gs://gemini-lavi-asset/production/earning_call...,earning_calls,blocked,"[0.044941700994968414, -0.006671776995062828, ..."
1,gs://gemini-lavi-asset/production/earning_call...,earning_calls,## Alphabet Q1 2023 Earnings Call Analysis \n\...,"[-0.0017111977795138955, -0.01606634259223938,..."
2,gs://gemini-lavi-asset/production/earning_call...,earning_calls,rategic direction.\n\n* **Analysis:**\nEmphasi...,"[-0.004538985434919596, -0.023720985278487206,..."
3,gs://gemini-lavi-asset/production/earning_call...,earning_calls,* **Summary:**\nThis segment focuses on AI's g...,"[0.017779091373085976, -0.002844201633706689, ..."
4,gs://gemini-lavi-asset/production/earning_call...,earning_calls,wards generative AI and large language models ...,"[-0.005946682300418615, -0.019808892160654068,..."


## Step 4 - Building Index

In [ ]:
# Step 4 Building final metadata and index for the vector db
# Building index

extracted_text_chunk_df['index'] = extracted_text_chunk_df['page_number'].astype(str)+ "_"+extracted_text_chunk_df['text_type']+ "_" + extracted_text_chunk_df.index.astype(str)
video_metadata_chunk_df['index'] = video_metadata_chunk_df['video_type']+ "_" + video_metadata_chunk_df.index.astype(str)
audio_metadata_chunk_df['index'] = audio_metadata_chunk_df['audio_type']+ "_" + audio_metadata_chunk_df.index.astype(str)

#Adding source to identify type of file
print("Adding source type in the metadata......")
extracted_text_chunk_df['source'] = "text_based"
video_metadata_chunk_df['source'] = "video_based"
audio_metadata_chunk_df['source'] = "audio_based"

#Building index data
print("Building index data from the metadata......")
index_db = pd.concat([extracted_text_chunk_df[['index','source','embeddings']],
          video_metadata_chunk_df[['index','source','embeddings']],
          audio_metadata_chunk_df[['index','source','embeddings']]
                      ],
        axis=0).reset_index(drop=True)

In [91]:
index_db.head()

,index,source,embeddings
0,1_blogpost_0,text_based,"[0.0026181433349847794, -0.006980935577303171,..."
1,2_blogpost_1,text_based,"[0.02464565820991993, -0.020231368020176888, -..."
2,3_blogpost_2,text_based,"[0.020969916135072708, 0.0036780794616788626, ..."
3,3_blogpost_3,text_based,"[0.012977363541722298, -0.02937842719256878, -..."
4,3_blogpost_4,text_based,"[0.052400510758161545, -0.06389233469963074, -..."


In [93]:
extracted_text_chunk_df.head()

,text_type,gcs_path,page_number,text,embeddings,index,source
0,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,1,Google\nCloud\ndemonstrates\nthe\nworld’ s\nla...,"[0.0026181433349847794, -0.006980935577303171,...",1_blogpost_0,text_based
1,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,2,Source:\n“Computation\nused\nto\ntrain\nnotabl...,"[0.02464565820991993, -0.020231368020176888, -...",2_blogpost_1,text_based
2,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,3,1.\nOrchestration:\nThe\nsoftware\nstack\nthat...,"[0.020969916135072708, 0.0036780794616788626, ...",3_blogpost_2,text_based
3,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,3,e\nscale\nrequires\ndeep\nexpertise\nthroughou...,"[0.012977363541722298, -0.02937842719256878, -...",3_blogpost_3,text_based
4,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,3,"scalable,\nreliable,\nand\neasy-to-use\nfor\ne...","[0.052400510758161545, -0.06389233469963074, -...",3_blogpost_4,text_based


In [94]:
video_metadata_chunk_df.head()

,video_gcs,video_type,video_description,embeddings,index,source
0,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,## Video Analysis: Understanding Multimodal Mo...,"[0.01595981791615486, -0.04110850393772125, -0...",product_launch/gemini_0,video_based
1,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,ike a puffy jacket establishes a baseline for ...,"[0.0016808598302304745, -0.02568306028842926, ...",product_launch/gemini_1,video_based
2,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,**Thematic Context**: Illustrates Gemini's a...,"[0.025427915155887604, -0.050456706434488297, ...",product_launch/gemini_2,video_based
3,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,\n\n**Segment 3: Introducing New Elements and...,"[0.03876801207661629, -0.024181410670280457, -...",product_launch/gemini_3,video_based
4,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,actical to imaginative responses underscores t...,"[0.04932378977537155, -0.03003864549100399, -0...",product_launch/gemini_4,video_based


In [95]:
audio_metadata_chunk_df.head()

,audio_gcs,audio_type,audio_description,embeddings,index,source
0,gs://gemini-lavi-asset/production/earning_call...,earning_calls,blocked,"[0.044941700994968414, -0.006671776995062828, ...",earning_calls_0,audio_based
1,gs://gemini-lavi-asset/production/earning_call...,earning_calls,## Alphabet Q1 2023 Earnings Call Analysis \n\...,"[-0.0017111977795138955, -0.01606634259223938,...",earning_calls_1,audio_based
2,gs://gemini-lavi-asset/production/earning_call...,earning_calls,rategic direction.\n\n* **Analysis:**\nEmphasi...,"[-0.004538985434919596, -0.023720985278487206,...",earning_calls_2,audio_based
3,gs://gemini-lavi-asset/production/earning_call...,earning_calls,* **Summary:**\nThis segment focuses on AI's g...,"[0.017779091373085976, -0.002844201633706689, ...",earning_calls_3,audio_based
4,gs://gemini-lavi-asset/production/earning_call...,earning_calls,wards generative AI and large language models ...,"[-0.005946682300418615, -0.019808892160654068,...",earning_calls_4,audio_based


### Backing up the metadata

In [ ]:
# backup_metadata_in_pickle(extracted_text,
#                           video_metadata,
#                           audio_metadata,
#                           index_db,
#                           output_path_with_name="/content/metadata"
#                           )

In [97]:
# download documents and images used in this notebook - will take ~30 sec
!gsutil -m -q rsync -r gs://gemini-lavi-asset/asset/ .
print("Download completed")


both the source and destination. Your crcmod installation isn't using the
module's C extension, so checksumming will run very slowly. If this is your
first rsync since updating gsutil, this rsync can take significantly longer than
usual. For help installing the extension, please see "gsutil help crcmod".

Download completed


In [5]:
# extracted_text, video_metadata, audio_metadata, index_db = load_backuped_metadata("/content/metadata")
import pickle
extracted_text, video_metadata, audio_metadata, index_db = load_backuped_metadata(bucket)

In [6]:
extracted_text.head()

,text_type,gcs_path,page_number,text,embeddings,index,source
0,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,1,Google\nCloud\ndemonstrates\nthe\nworld’ s\nla...,"[0.0026181433349847794, -0.006980935577303171,...",1_blogpost_0,text_based
1,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,2,Source:\n“Computation\nused\nto\ntrain\nnotabl...,"[0.02464565820991993, -0.020231368020176888, -...",2_blogpost_1,text_based
2,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,3,1.\nOrchestration:\nThe\nsoftware\nstack\nthat...,"[0.020969916135072708, 0.0036780794616788626, ...",3_blogpost_2,text_based
3,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,3,e\nscale\nrequires\ndeep\nexpertise\nthroughou...,"[0.012977363541722298, -0.02937842719256878, -...",3_blogpost_3,text_based
4,blogpost,gs://gemini-lavi-asset/production/blogpost/Goo...,3,"scalable,\nreliable,\nand\neasy-to-use\nfor\ne...","[0.052400510758161545, -0.06389233469963074, -...",3_blogpost_4,text_based


In [103]:
video_metadata.head()

,video_gcs,video_type,video_description,embeddings,index,source
0,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,## Video Analysis: Understanding Multimodal Mo...,"[0.01595981791615486, -0.04110850393772125, -0...",product_launch/gemini_0,video_based
1,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,ike a puffy jacket establishes a baseline for ...,"[0.0016808598302304745, -0.02568306028842926, ...",product_launch/gemini_1,video_based
2,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,**Thematic Context**: Illustrates Gemini's a...,"[0.025427915155887604, -0.050456706434488297, ...",product_launch/gemini_2,video_based
3,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,\n\n**Segment 3: Introducing New Elements and...,"[0.03876801207661629, -0.024181410670280457, -...",product_launch/gemini_3,video_based
4,gs://gemini-lavi-asset/production/product_laun...,product_launch/gemini,actical to imaginative responses underscores t...,"[0.04932378977537155, -0.03003864549100399, -0...",product_launch/gemini_4,video_based


In [11]:
audio_metadata.head()

,audio_gcs,audio_type,audio_description,embeddings,index,source
0,gs://gemini-lavi-asset/production/earning_call...,earning_calls,blocked,"[0.044941700994968414, -0.006671776995062828, ...",earning_calls_0,audio_based
1,gs://gemini-lavi-asset/production/earning_call...,earning_calls,## Alphabet Q1 2023 Earnings Call Analysis \n\...,"[-0.0017111977795138955, -0.01606634259223938,...",earning_calls_1,audio_based
2,gs://gemini-lavi-asset/production/earning_call...,earning_calls,rategic direction.\n\n* **Analysis:**\nEmphasi...,"[-0.004538985434919596, -0.023720985278487206,...",earning_calls_2,audio_based
3,gs://gemini-lavi-asset/production/earning_call...,earning_calls,* **Summary:**\nThis segment focuses on AI's g...,"[0.017779091373085976, -0.002844201633706689, ...",earning_calls_3,audio_based
4,gs://gemini-lavi-asset/production/earning_call...,earning_calls,wards generative AI and large language models ...,"[-0.005946682300418615, -0.019808892160654068,...",earning_calls_4,audio_based


In [12]:
index_db.head()

,index,source,embeddings
0,1_blogpost_0,text_based,"[0.0026181433349847794, -0.006980935577303171,..."
1,2_blogpost_1,text_based,"[0.02464565820991993, -0.020231368020176888, -..."
2,3_blogpost_2,text_based,"[0.020969916135072708, 0.0036780794616788626, ..."
3,3_blogpost_3,text_based,"[0.012977363541722298, -0.02937842719256878, -..."
4,3_blogpost_4,text_based,"[0.052400510758161545, -0.06389233469963074, -..."


In [8]:
print(gemini_15_pro_new.count_tokens('-'.join(extracted_text['text'])))

total_tokens: 1434876
total_billable_characters: 4298035



In [9]:
print(gemini_15_pro_new.count_tokens('-'.join(video_metadata['video_description'])))

total_tokens: 16778
total_billable_characters: 71901



In [10]:
print(gemini_15_pro_new.count_tokens('-'.join(audio_metadata['audio_description'])))

total_tokens: 21152
total_billable_characters: 93818



## Retreival & Generation

In [14]:
#@title Helper Functions

def get_cosine_score(
    dataframe: pd.DataFrame, column_name: str, input_text_embd: np.ndarray
) -> float:
    """
    Calculates the cosine similarity between the user query embedding and the dataframe embedding for a specific column.

    Args:
        dataframe: The pandas DataFrame containing the data to compare against.
        column_name: The name of the column containing the embeddings to compare with.
        input_text_embd: The NumPy array representing the user query embedding.

    Returns:
        The cosine similarity score (rounded to two decimal places) between the user query embedding and the dataframe embedding.
    """
    if dataframe[column_name]:
      text_cosine_score = round(np.dot(dataframe[column_name], input_text_embd), 2)
      return text_cosine_score
    else:
      return 0

def get_timestamp_with_milliseconds():
    """Gets the current time as a timestamp string with milliseconds."""

    now = datetime.now(timezone.utc)  # Get current UTC time with timezone awareness
    timestamp_str = now.strftime("%Y-%m-%d-%H:%M:%S.%f")[:-3]  # Format with milliseconds

    return timestamp_str

def get_pdf_from_matched_index(text_index):
  # print("Unique Text index: ", unique_text_cit)
  storage_client = storage.Client(project='kaggle-on-gcp')
  bucket = storage_client.bucket("gemini-lavi-asset")

  selected_file = {}
  gcs_path = extracted_text[extracted_text['index']==text_index]['gcs_path'].values[0].split("//")[1]
  gcs_path = "/".join(gcs_path.split("/")[1:])
  page_num = extracted_text[extracted_text['index']==text_index]['page_number'].values[0]
  selected_file[gcs_path] = int(page_num)
  # print(selected_file)
  pdf_object_part_list = []
  for blob in bucket.list_blobs(prefix="production/"):
      if blob.name in selected_file:
        pdf_content = BytesIO(blob.download_as_bytes())
        pdf_reader = PyPDF2.PdfReader(pdf_content)
        pdf_writer = PyPDF2.PdfWriter()
        pdf_writer.add_page(pdf_reader.pages[selected_file[blob.name]])

        # pdf_writer.add_page(pdf_reader.pages[selected_file[blob.name]])
        # Create a BytesIO buffer to hold the output PDF
        output_buffer = BytesIO()

        # Write the PDF to the buffer
        pdf_writer.write(output_buffer)
        output_buffer.seek(0)  # Reset buffer position to the beginning

        # Upload the buffer content to GCS
        new_blob_name = f"""temp/{blob.name.split("/")[-1]}_selected_pages_{selected_file[blob.name]}_{get_timestamp_with_milliseconds()}.pdf"""
        new_blob = bucket.blob(new_blob_name)
        # print(new_blob_name)
        # print(new_blob)
        new_blob.upload_from_string(output_buffer.getvalue(), content_type="application/pdf")
        gcs_path = "gs://"+blob.bucket.name+"/"+new_blob_name
        pdf_object_part_list.extend(["filename: ",gcs_path, Part.from_uri(uri=gcs_path, mime_type="application/pdf")])
        # print(f"Uploaded selected pages to: {new_blob_name}")

  # print("Created temp pdf's to address the query in GCS.....")
  return pdf_object_part_list

def get_gemini_content_list(query, vector_db, top_n_cosine_scores):
  instruction = """Task: Answer the following questions in detail, providing clear reasoning and evidence from the context files in bullet points.
  Instructions:

  1. **Analyze:** Carefully examine the provided images and text context.
  2. **Synthesize:** Integrate information from both the visual and textual elements.
  3. **Reason:**  Deduce logical connections and inferences to address the question.
  4. **Respond:** Provide a concise, accurate answer in the following format:

    * **Question:** [Question]
    * **Answer:** [Direct response to the question]
    * **Explanation:** [Bullet-point reasoning steps if applicable]
    * **Source** [name of the file, page, image from where the information is citied]

  5. **Ambiguity:** If the context is insufficient to answer, respond "Not enough context to answer."
  """
  gemini_content = [instruction,"Questions:", query,
  "Contexual Files:" ]

  vector_db_sample = vector_db.iloc[top_n_cosine_scores]
  uri_track = []
  for index, row in vector_db_sample.iterrows():
    if row['source'] == 'video_based':
      gcs_path = video_metadata[video_metadata['index']==row['index']]['video_gcs'].values[0]
      mime = 'video/mp4'
      if gcs_path not in uri_track:
        uri_track.append(gcs_path)
        gemini_content.extend(["filename: ",gcs_path,Part.from_uri(uri=gcs_path, mime_type= mime)])

    elif row['source'] == 'audio_based':
      gcs_path = audio_metadata[audio_metadata['index']==row['index']]['audio_gcs'].values[0]
      mime = 'audio/mpeg'
      if gcs_path not in uri_track:
        uri_track.append(gcs_path)
        gemini_content.extend(["filename: ",gcs_path, Part.from_uri(uri=gcs_path, mime_type= mime)])

    elif row['source'] == 'text_based':
      pdf_object_part_list = get_pdf_from_matched_index(row['index'])
      # print("Some files are temp uploaded to gcs to support the query....")
      gemini_content.extend(pdf_object_part_list)

    else:
      print ("Something has gone wrong......")

  return gemini_content

def get_gemini_response(model, generation_config=None,
                        safety_settings=None,
                        uri_path=None,mime_type=None, prompt=None):
  if not generation_config:
    generation_config = {
      "max_output_tokens": 8192,
      "temperature": 1,
      "top_p": 0.95,
    }

  if not safety_settings:
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

  responses = model.generate_content(prompt,
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )
  final_response = []
  for response in responses:
    try:
      final_response.append(response.text)
    except ValueError:
      # print("Something is blocked...")
      final_response.append("blocked")

  return "".join(final_response)

def get_answer(query,vector_db, model, top_n=5):
  query_embedding = get_text_embeddings([query])
  #Find score
  cosine_scores = vector_db.apply(
              lambda x: get_cosine_score(x, 'embeddings', query_embedding),
              axis=1,
          )
  # Remove same image comparison score when user image is matched exactly with metadata image
  # cosine_scores = cosine_scores[cosine_scores < 1.00000000]
  # Get top N cosine scores and their indices
  top_n_cosine_scores = cosine_scores.nlargest(top_n).index.tolist()
  top_n_cosine_values = cosine_scores.nlargest(top_n).values.tolist()

  citations = vector_db.iloc[top_n_cosine_scores]
  # citations['score'] = top_n_cosine_scores
  citations.loc[:, 'score'] = top_n_cosine_values
  citations = citations[['index','source','score']]

  # print(citations)
  gemini_content = get_gemini_content_list(query, vector_db, top_n_cosine_scores)

  response  =  get_gemini_response(model=model, prompt=gemini_content)
  return([response, gemini_content, citations])

In [126]:
query = """- What is the role of AI in accelerating the progress of UN's sustainable development goals
- What specific commitments and initiatives demonstrate Google's collaboration with the UN on AI for good?
"""

In [127]:
%%time
response, gemini_content, citation = get_answer(query, index_db, gemini_15_pro_new,top_n=5)
rich_Markdown(response)

CPU times: user 2.11 s, sys: 115 ms, total: 2.22 s
Wall time: 54.6 s


 • Question: What is the role of AI in accelerating the progress of UN's sustainable development goals?            
 • Answer: AI can accelerate progress on UN's SDGs by:                                                             
    • Modeling protein structures: AI, specifically AlphaFold, can model protein structures which helps in the     
      development of vaccines and disease research. This can contribute to achieving Goal 3 (Good Health and       
      Wellbeing). (Source: gs://gemini-lavi-asset/production/podcast/dialogue_dispatch/How can AI help accelerate  
      progress on the UN SDGs_ _ Dialogues Dispatch Podcast _ Episode 6.mp3)                                       
    • Improving food security: AI can be used to reduce crop loss due to pests and diseases, contributing to Goal 2
      (Zero Hunger). (Source: gs://gemini-lavi-asset/production/podcast/dialogue_dispatch/How can AI help          
      accelerate progress on the UN SDGs_ _ Dialogues Dispatch Podcast _ Episode 6.mp3)                            
    • Predicting weather events: AI-powered systems can predict weather events like floods with increased accuracy,
      helping communities prepare and mitigate risks related to Goal 13 (Climate Action). (Source:                 
      gs://gemini-lavi-asset/production/podcast/dialogue_dispatch/How can AI help accelerate progress on the UN    
      SDGs_ _ Dialogues Dispatch Podcast _ Episode 6.mp3)                                                          
    • Data accessibility: AI makes data related to SDG progress more accessible through platforms like Google's    
      Data Commons, aiding in monitoring and evaluating progress towards the goals. (Source:                       
      gs://gemini-lavi-asset/production/podcast/dialogue_dispatch/How can AI help accelerate progress on the UN    
      SDGs_ _ Dialogues Dispatch Podcast _ Episode 6.mp3)                                                          
    • Attracting talent and innovation: The potential of AI in solving global challenges can attract a new         
      generation of talent and drive innovations towards achieving the SDGs. (Source:                              
      gs://gemini-lavi-asset/production/podcast/dialogue_dispatch/How can AI help accelerate progress on the UN    
      SDGs_ _ Dialogues Dispatch Podcast _ Episode 6.mp3)                                                          
 • Question: What specific commitments and initiatives demonstrate Google's collaboration with the UN on AI for    
   good?                                                                                                           
 • Answer: Google demonstrates its commitment to AI for good through these collaborations:                         
    • Data Commons: Google collaborates with organizations like the UN to make data sets related to the SDGs       
      publicly accessible through Data Commons. (Source:                                                           
      gs://gemini-lavi-asset/production/podcast/dialogue_dispatch/How can AI help accelerate progress on the UN    
      SDGs_ _ Dialogues Dispatch Podcast _ Episode 6.mp3)                                                          
    • Commitments at the White House and AI Safety Summit: Google has made commitments at the White House and      
      participated in the AI Safety Summit, collaborating with global entities to discuss and ensure the           
      responsible development of AI. (Source: gs://gemini-lavi-asset/production/podcast/dialogue_dispatch/How can  
      AI help accelerate progress on the UN SDGs_ _ Dialogues Dispatch Podcast _ Episode 6.mp3)                    
    • Election Integrity Efforts: Google works with election authorities globally to fight misinformation and      
      protect election integrity using AI tools. (Source:                                                          
      gs://gemini-lavi-asset/production/podcast/dialogue

In [130]:
%%time
response, gemini_content, citation = get_answer(query, index_db, gemini_15_flash,top_n=5)
rich_Markdown(response)

CPU times: user 2.08 s, sys: 73.2 ms, total: 2.16 s
Wall time: 33.6 s


 • Question: What is the role of AI in accelerating the progress of UN's sustainable development goals?            
 • Answer: AI can play a significant role in accelerating progress toward the UN's Sustainable Development Goals by
   providing data insights, supporting innovation, and enabling greater efficiency.                                
 • Explanation:                                                                                                    
    • AI can analyze vast amounts of data to identify trends and patterns that inform decision-making for achieving
      the SDGs.                                                                                                    
    • AI can support the development of innovative solutions for addressing critical challenges related to health, 
      education, and other areas.                                                                                  
    • AI can streamline processes, optimize resource allocation, and improve efficiency in various sectors.        
 • Source:  gs://gemini-lavi-asset/production/podcast/dialogue_dispatch/How can AI help accelerate progress on the 
   UN SDGs_ _ Dialogues Dispatch Podcast _ Episode 6.mp3 (00:01 – 19:32)                                           
 • Question: What specific commitments and initiatives demonstrate Google's collaboration with the UN on AI for    
   good?                                                                                                           
 • Answer: Google has partnered with the UN on initiatives like Data Commons, which makes data more accessible for 
   analysis, and AlphaFold, which contributes to scientific breakthroughs in areas like disease research and food  
   security.                                                                                                       
 • Explanation:                                                                                                    
    • Google has contributed to the UN's Data Commons project by providing technical expertise and resources to    
      make data more accessible and usable for analysis related to the SDGs.                                       
    • Google's AI research team, DeepMind, developed AlphaFold, a groundbreaking AI system that predicts protein   
      structures, accelerating research in areas like disease treatment and food security.                         
 • Source: gs://gemini-lavi-asset/production/podcast/dialogue_dispatch/How can AI help accelerate progress on the  
   UN SDGs_ _ Dialogues Dispatch Podcast _ Episode 6.mp3 (00:01 – 19:32)

In [128]:
gemini_content

['Task: Answer the following questions in detail, providing clear reasoning and evidence from the context files in bullet points.\n  Instructions:\n\n  1. **Analyze:** Carefully examine the provided images and text context.\n  2. **Synthesize:** Integrate information from both the visual and textual elements.\n  3. **Reason:**  Deduce logical connections and inferences to address the question.\n  4. **Respond:** Provide a concise, accurate answer in the following format:\n\n    * **Question:** [Question]\n    * **Answer:** [Direct response to the question]\n    * **Explanation:** [Bullet-point reasoning steps if applicable]\n    * **Source** [name of the file, page, image from where the information is citied]\n\n  5. **Ambiguity:** If the context is insufficient to answer, respond "Not enough context to answer."\n  ',
 'Questions:',
 "- What is the role of AI in accelerating the progress of UN's sustainable development goals\n- What specific commitments and initiatives demonstrate Goog

In [129]:
citation

,index,source,score
11461,podcast/dialogue_dispatch_122,audio_based,0.84
11278,product_launch/gemini_116,video_based,0.83
11447,podcast/dialogue_dispatch_108,audio_based,0.83
11442,podcast/dialogue_dispatch_103,audio_based,0.82
11455,podcast/dialogue_dispatch_116,audio_based,0.82


In [15]:
%%time

# time.sleep(30)
query = "How does Gemini 1.5 long context works with video, images, text and code? Give detail examples that Google showed?"
response, gemini_content, citation = get_answer(query, index_db, gemini_15_pro_new,top_n=5)
rich_Markdown(response)

CPU times: user 2.12 s, sys: 168 ms, total: 2.28 s
Wall time: 1min 7s


Question: How does Gemini 1.5 long context works with video, images, text and code? Give detail examples that      
Google showed?                                                                                                     

Answer: Gemini 1.5's long context window leverages a vast 1 million multimodal tokens, allowing it to effectively  
process and integrate information from video, images, text, and code.                                              

Explanation:                                                                                                       

 • Code Comprehension and Modification:  Gemini 1.5 can analyze extensive codebases (e.g., 100,000 lines of        
   JavaScript code) and respond to prompts for identifying specific code examples and modifying them based on user 
   requests. In the demonstration, it accurately identified code snippets related to animation blending, poses, and
   morph targets. It also successfully added a GUI slider to control animation speed and modified terrain          
   generation code to make the terrain flatter. (Source: Problem solving across 100633 lines of code  Gemini 15 Pro
   Demo.mp4)                                                                                                       
 • Text Analysis and Extraction: Gemini 1.5 can process lengthy text documents (e.g., a 402-page Apollo 11         
   transcript) to identify key moments and extract relevant information. In the demo, it successfully found comedic
   moments from the transcript and provided accurate quotes along with corresponding timecodes. (Source: Reasoning 
   across a 402-page transcript  Gemini 15 Pro Demo.mp4)                                                           
 • Video Understanding and Scene Identification:  Gemini 1.5 can analyze videos (e.g., a 44-minute Buster Keaton   
   film) and accurately identify specific scenes based on user prompts. In the demonstration, it located the exact 
   moment a pawn ticket was removed from a person's pocket and provided the corresponding timecode. (Source:       
   Multimodal prompting with a 44-minute movie  Gemini 15 Pro Demo.mp4)                                            
 • Multimodal Prompting:  Gemini 1.5 effectively integrates visual and textual information to respond to complex   
   prompts. It successfully identified Neil Armstrong's first steps on the moon from a crude drawing and associated
   text prompt, demonstrating its ability to handle abstract visual cues and relate them to the context of a large 
   text document. Further, it accurately identified a scene from the Buster Keaton film based on a simple drawing  
   and a text prompt requesting the corresponding timecode. (Source: Reasoning across a 402-page transcript  Gemini
   15 Pro Demo.mp4, Multimodal prompting with a 44-minute movie  Gemini 15 Pro Demo.mp4)                           

Note: Google emphasizes that the long context feature in Gemini 1.5 is experimental and processing times can vary. 
While the responses are not always perfect, they demonstrate the potential of Gemini 1.5 in handling large,        
multimodal inputs.

In [16]:
citation

,index,source,score
11251,product_launch/gemini_89,video_based,0.82
11259,product_launch/gemini_97,video_based,0.82
11266,product_launch/gemini_104,video_based,0.81
11238,product_launch/gemini_76,video_based,0.79
11233,product_launch/gemini_71,video_based,0.78


In [17]:
%%time

# time.sleep(30)
query = "What are key achievement for Google Cloud in terms of training LLMs using their TPUs?"
response, gemini_content, citation = get_answer(query, index_db, gemini_15_pro_new,top_n=5)
rich_Markdown(response)

CPU times: user 2.42 s, sys: 137 ms, total: 2.56 s
Wall time: 16.8 s


Question: What are key achievements for Google Cloud in terms of training LLMs using their TPUs?                   

Answer: Google Cloud has achieved robust orchestration and scalability for training LLMs using TPUs, enabling      
training across tens of thousands of TPU chips with reliability and fault tolerance. They also ensure performant   
compilation by leveraging the XLA compiler for efficient compute and communication management. Furthermore, they   
offer a flexible stack for end-to-end training, supporting frameworks like JAX and PyTorch, and providing reference
implementations and libraries for various model architectures like LLMs.                                           

Explanation:                                                                                                       

 • Google Cloud's TPU blog highlights three key components of their Multislice Training stack: Robust orchestration
   and scalability, Performant compilation, and Flexible stack for end-to-end training.                            
 • This stack enables scaling large model training across a massive number of TPU chips while maintaining          
   reliability.                                                                                                    
 • The XLA compiler maximizes performance and efficiency in compute and communication.                             
 • The flexible stack allows for easy integration with popular ML frameworks and provides comprehensive support for
   various model architectures, including LLMs.                                                                    

Source: "gs://gemini-lavi-asset/temp/Google Cloud TPU blog.pdf_selected_pages_4_2024-05-15-14:33:56.104.pdf" (Page 
1)

In [18]:
citation

,index,source,score
5,4_blogpost_5,text_based,0.84
12,6_blogpost_12,text_based,0.82
0,1_blogpost_0,text_based,0.81
10,6_blogpost_10,text_based,0.80
1,2_blogpost_1,text_based,0.79


In [19]:
%%time

# time.sleep(30)
query = "what is the emfu for bf16 and 128b parameter model with 1 tpu v5e pod? Cite the table and page number and explain the significance of the results"
response, gemini_content, citation = get_answer(query, index_db, gemini_15_pro_new,top_n=5)
rich_Markdown(response)

CPU times: user 3.03 s, sys: 132 ms, total: 3.16 s
Wall time: 14.5 s


 • Question: what is the emfu for bf16 and 128b parameter model with 1 tpu v5e pod? Cite the table and page number 
   and explain the significance of the results                                                                     
 • Answer:  The EMFU for a 128B parameter model with BF16 and a single TPU v5e pod is 56.06%.                      
 • Explanation:                                                                                                    
    • The EMFU (ExaFLOPS/s Model FLOPS Utilization) represents the efficiency of utilizing the hardware's          
      computational power for the specific model.                                                                  
    • This information can be found in the "MaxText LLM Training Results" table on page 13 of the document.        
 • Source: "Google Cloud TPU blog.pdf_selected_pages_13_2024-05-15-14:35:25.508.pdf", Page 13

In [20]:
citation

,index,source,score
28,12_blogpost_28,text_based,0.78
29,13_blogpost_29,text_based,0.78
12,6_blogpost_12,text_based,0.75
4340,117_reports/2021/annual_report_4340,text_based,0.75
17,8_blogpost_17,text_based,0.74
